In [3]:
def cnn_trts(tr,trl,ts,tsl,lr):
    #调用已写好的神经网络统计函数,已经包含tensorflow
    %run nn_functions.ipynb

    #需要执行此语句，否则之后重复运行会产生报错，这是针对jupyter的解决方案
    tf.reset_default_graph()

    batchsize=100  #根据数据集进行的调整
    nsize=tr.shape[0]//batchsize
    
    #将标签转为独热编码
    from sklearn import preprocessing
    enc = preprocessing.OneHotEncoder()
    enc.fit(tsl)

    trl=enc.transform(trl).toarray()
    tsl=enc.transform(tsl).toarray()
    
    nf=64 #特征数量
    c1=32 #第一层卷积数量
    c2=64 #第二层卷积数量
    t1=512 #第一层全连接层神经元数
    t2=8   #第二层全连接层神经元数
    
    #创建占位符
    with tf.name_scope('input'):
        x=tf.placeholder(tf.float32,[None,nf],name='xinput')
        y=tf.placeholder(tf.float32,[None,t2],name='yinput')
    with tf.name_scope('label'):
        ptl=tf.placeholder(tf.float32,[None,t2],name='ptl')
        ptl_prob=tf.placeholder(tf.float32,[None,t2],name='ptlprob')

    #定义卷积层函数，这里w是滤波器，[5,5,1,32]意味用5*5的卷积核从1个平面抽取32个特征
    def convd2(x,w):
        return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

    #定义池化层函数
    def max_pool_22(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')#ksize为窗口大小2*2,strides为步长，目前0,3两个位置1固定

    #将一维数组转为二维（图像）
    x_4d=tf.reshape(x,[-1,8,8,1])#这里0位置等于-1代表行数任意，这里为batchsize，3位置等于1代表输入通道为1（黑白）


    with tf.name_scope('convol'):   
        with tf.name_scope('convol1'):
            #初始化第一层卷积层权值和偏置值
            w_conv1=weight_variable([3,3,1,c1])
            b_conv1=bias_variable([c1])

            #得到第一层卷积层并池化
            conv1=tf.nn.relu(convd2(x_4d,w_conv1)+b_conv1)
            pconv1=max_pool_22(conv1)
        with tf.name_scope('convol2'):
            #初始化第二层卷积层的权值和偏置值
            w_conv2=weight_variable([3,3,c1,c2])
            b_conv2=bias_variable([c2])

            #得到第二层卷积层并池化
            conv2=tf.nn.relu(convd2(pconv1,w_conv2)+b_conv2)
            pconv2=max_pool_22(conv2)

    #将最后一层池化层展平后建立双层全连接网络
    p_flat=tf.reshape(pconv2,[-1,2*2*c2])

    #采用dropout方式建立全连接层
    with tf.name_scope('layer'):
        with tf.name_scope('layer1'):
            w1=weight_variable([2*2*c2,t1])
            b1=bias_variable([t1])
            wb1=tf.matmul(p_flat,w1)+b1
            op1=tf.nn.relu(wb1)
            trainpercent=tf.placeholder(tf.float32)
            op1train=tf.nn.dropout(wb1,trainpercent)

        with tf.name_scope('layer2'):
            w2=weight_variable([t1,t2])
            b2=bias_variable([t2])
            wb2=tf.matmul(op1train,w2)+b2
            op2=tf.nn.softmax(wb2)
            prediction=wb2

        
    #定义损失以及训练优化器
    with tf.name_scope('loss'):
        loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
        #loss=tf.reduce_mean(tf.reduce_sum(tf.square(y-prediction),reduction_indices=[1]))
        #loss=tf.reduce_mean(tf.square(y-prediction))
        tf.summary.scalar('loss',loss)
    with tf.name_scope('train'):
        Trainstep=tf.train.AdamOptimizer(lr).minimize(loss)
       
    
    with tf.name_scope('result'):
        
        ptl=tf.argmax(prediction,1) #返回训练标签
        ptl_prob=tf.reduce_max(prediction,reduction_indices=[1])    


    with tf.name_scope('acc'):
        with tf.name_scope('correctprediction'):
            correctprediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
        with tf.name_scope('accuracy'):
            accuracy=tf.reduce_mean(tf.cast(correctprediction,tf.float32))
            tf.summary.scalar('accuracy',accuracy)


    #合并summary
    merge=tf.summary.merge_all()

    #训练神经网络
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        writer=tf.summary.FileWriter('logs',sess.graph)

        for epoch in range(1024):
            istart=0
            iend=batchsize

            for i in range(nsize):
                #sess.run(tf.assign(lr,0.001*(0.97**epoch)))
                #bx,by=mnist.train.next_batch(batchsize)
                
                bx=tr[istart:iend,:]
                by=trl[istart:iend,:]
                statistic,_=sess.run([merge,Trainstep],feed_dict={x:bx,y:by,trainpercent:1.})
                istart=iend
                iend=istart+batchsize
            #将summary的内容写入writer中
            writer.add_summary(statistic,epoch)

            rate=sess.run(accuracy,feed_dict={x:tr,y:trl,trainpercent:1.})
            print(str(rate))
        ptr=sess.run(ptl,feed_dict={x:tr,y:trl,trainpercent:1.})
        pts=sess.run(ptl,feed_dict={x:ts,y:tsl,trainpercent:1.})
        ptr_prob=sess.run(ptl_prob,feed_dict={x:tr,y:trl,trainpercent:1.})
        pts_prob=sess.run(ptl_prob,feed_dict={x:ts,y:tsl,trainpercent:1.})
    return ptr,ptr_prob,pts,pts_prob